https://text2sparql.aksw.org/challenge/


In [1]:
endpoint = "https://wse-research.org/kgqagent/api"

In [ ]:
# works
question = """Which products are categorized as Hardware and what physical dimensions (height, width, depth) can be stored for them?"""

sparql="""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX pv: <http://ld.company.org/prod-vocab/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?product ?productName ?height ?width ?depth ?weight
WHERE {
  # Find products that are of type Hardware
  ?product rdf:type pv:Hardware .
  
  # Get the product name if available
  OPTIONAL { ?product pv:name ?productName }
  
  # Get physical dimensions if available
  OPTIONAL { ?product pv:height_mm ?height }
  OPTIONAL { ?product pv:width_mm ?width }
  OPTIONAL { ?product pv:depth_mm ?depth }
  OPTIONAL { ?product pv:weight_g ?weight }
}
ORDER BY ?productName"""

In [ ]:
# works
question = """How many suppliers are there in the data model?"""
question = "How many suppliers exist?"
sparql = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX pv: <http://ld.company.org/prod-vocab/>

SELECT (COUNT(DISTINCT ?supplier) AS ?supplierCount)
WHERE {
  ?supplier rdf:type pv:Supplier .
} """

In [ ]:
# works
question = "How members has each department?"

sparql = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX pv: <http://ld.company.org/prod-vocab/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?department ?departmentName (COUNT(DISTINCT ?member) AS ?memberCount)
WHERE {
  # Find all departments
  ?department rdf:type pv:Department .
  
  # Get department name if available
  OPTIONAL { ?department pv:name ?departmentName }
  
  # Find all members of this department
  OPTIONAL {
    ?member pv:memberOf ?department .
  }
}
GROUP BY ?department ?departmentName
ORDER BY DESC(?memberCount)
"""

In [ ]:
question = "What contact information can be stored for Agents (including Employees) in this knowledge graph?"
sparql = """

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX pv: <http://ld.company.org/prod-vocab/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

# Find all properties that can be used for contact information for Agents
SELECT DISTINCT ?contactProperty ?propertyLabel ?propertyComment ?range
WHERE {
  # Find properties with domain pv:Agent
  ?contactProperty rdfs:domain pv:Agent .
  
  # Get label and comment for the property
  OPTIONAL { ?contactProperty rdfs:label ?propertyLabel }
  OPTIONAL { ?contactProperty rdfs:comment ?propertyComment }
  
  # Get range (data type) of the property
  OPTIONAL { ?contactProperty rdfs:range ?range }
  
  # Ensure we only get properties that might be considered contact information
  # This filter looks for properties that might be contact related
  FILTER(
    # Include properties with specific names likely to be contact info
    CONTAINS(STR(?contactProperty), "email") || 
    CONTAINS(STR(?contactProperty), "phone") || 
    CONTAINS(STR(?contactProperty), "address") ||
    # Properties that are explicitly labeled as contact info
    (BOUND(?propertyComment) && (
      CONTAINS(LCASE(STR(?propertyComment)), "contact") ||
      CONTAINS(LCASE(STR(?propertyComment)), "email") ||
      CONTAINS(LCASE(STR(?propertyComment)), "phone") ||
      CONTAINS(LCASE(STR(?propertyComment)), "address")
    ))
  )
}
ORDER BY ?propertyLabel

"""

In [ ]:
# works
question = """What information is captured about Suppliers, and how are they connected to Products?"""

sparql="""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX pv: <http://ld.company.org/prod-vocab/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?supplier ?supplierName ?country ?countryName 
       ?email ?phone ?addressText ?addressLocality ?addressCountry ?addressCountryCode
       ?product ?productName ?productCategory ?categoryName
WHERE {
  # Find suppliers
  ?supplier rdf:type pv:Supplier .
  
  # Get supplier name if available
  OPTIONAL { ?supplier pv:name ?supplierName }
  
  # Get country information
  OPTIONAL { 
    ?supplier pv:country ?country .
    OPTIONAL { ?country rdfs:label ?countryName }
  }
  
  # Get contact information
  OPTIONAL { ?supplier pv:email ?email }
  OPTIONAL { ?supplier pv:phone ?phone }
  OPTIONAL { ?supplier pv:addressText ?addressText }
  OPTIONAL { ?supplier pv:addressLocality ?addressLocality }
  OPTIONAL { ?supplier pv:addressCountry ?addressCountry }
  OPTIONAL { ?supplier pv:addressCountryCode ?addressCountryCode }
  
  # Get products supplied by this supplier
  OPTIONAL {
    ?product pv:hasSupplier ?supplier .
    ?product rdf:type ?productType .
    FILTER(?productType = pv:Product || ?productType = pv:Hardware || ?productType = pv:Service)
    
    # Get product details
    OPTIONAL { ?product pv:name ?productName }
    OPTIONAL { 
      ?product pv:hasCategory ?productCategory .
      OPTIONAL { ?productCategory pv:name ?categoryName }
    }
  }
}
ORDER BY ?supplierName ?productName
"""

In [ ]:
# fails
question = """How are Product Managers connected to Products in the data model, and what is their relationship to other Employees?
"""
sparql = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX pv: <http://ld.company.org/prod-vocab/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?product ?productName ?productManager ?managerName ?department ?departmentName ?manager ?managerName ?directReport ?directReportName
WHERE {
  # Find products with their product managers
  ?product rdf:type pv:Product .
  ?product pv:hasProductManager ?productManager .
  
  # Get names where available
  OPTIONAL { ?product pv:name ?productName }
  OPTIONAL { ?productManager pv:name ?managerName }
  
  # Get department information for the product manager
  OPTIONAL { 
    ?productManager pv:memberOf ?department .
    OPTIONAL { ?department pv:name ?departmentName }
  }
  
  # If product manager has a manager
  OPTIONAL {
    ?productManager pv:hasManager ?manager .
    OPTIONAL { ?manager pv:name ?managerName }
  }
  
  # If product manager is a manager with direct reports
  OPTIONAL {
    ?productManager rdf:type pv:Manager .
    ?productManager pv:hasDirectReport ?directReport .
    OPTIONAL { ?directReport pv:name ?directReportName }
  }
}
ORDER BY ?productName ?managerName"""